<font size=6>**TD 3 : plongement de mots avec spacy**</font>

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

Ce notebook est juste une petite démonstration de l'utilisation des plongements de mots (statiques) à l'aide de la libraire spacy.

Au préalable, il faut (bien sûr) avoir installé la [librairie spacy](https://spacy.io/usage) mais il faut également penser à télécharger l'ensemble des vecteurs associés, ce qu'on appelle la version "large" de la base de données (cf. le "lg" à la fin).

Commande à lancer en console :

*python -m spacy download fr_core_news_lg*

In [5]:
# librairies
import spacy
import numpy as np

In [1]:
# on charge la ressource linguistique
nlp = spacy.load('fr_core_news_lg')

La fonction *find_close_words* cherche les mots les plus similaires dans l'espace vectoriel des plongements.

In [6]:
def find_close_words(word):
    sim_words = nlp.vocab.vectors.most_similar(
        np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word]]]), n=10)
    return [nlp.vocab.strings[w] for w in sim_words[0][0]]

# attention, nlp.vocab.strings fonctionne dans les deux sens :
#  - retourne l'identifiant (unique) correspondant à un mot
#  - retourne le mot correspondant à un identifiant

In [7]:
find_close_words("roi")
#find_close_words("café")
#find_close_words("deux")

['roi',
 'Roi',
 'prince',
 'monarque',
 'empereur',
 'régent',
 'Empereur',
 'suzerain',
 'coempereur',
 'l\x92empereur']

On peut également calculer des similarités entre mots grâce à la fonction *similarity* :

In [8]:
print("entre roi et reine : {}".format(nlp("roi").similarity(nlp("reine"))))
print("entre roi et trône : {}".format(nlp("roi").similarity(nlp("trône"))))
print("entre roi et oiseau : {}".format(nlp("roi").similarity(nlp("oiseau"))))

entre roi et reine : 0.628108097894871
entre roi et trône : 0.6489303722402299
entre roi et oiseau : 0.10113929594015392


Les vecteurs associés aux mots sont directement accessibles :

In [9]:
nlp("roi").vector

array([ 5.6351e+00, -4.7504e+00,  2.9866e+00, -6.5048e-02,  5.6314e+00,
       -2.6488e+00,  1.6728e+00,  3.6799e+00,  2.2033e+00, -2.3401e-01,
        1.7170e+00, -3.0931e+00,  3.8267e+00,  1.8726e-01, -2.2761e+00,
       -2.1582e+00,  3.8331e+00, -1.1703e-01,  1.2575e+00,  3.2968e+00,
       -1.1991e+00,  5.0724e-01, -2.6204e+00,  2.1288e+00,  9.9307e-01,
       -3.1832e+00, -2.3930e+00,  1.3869e+00, -1.7312e+00,  4.5163e+00,
        9.9608e-01, -3.1276e+00, -1.5539e+00,  1.0525e+00, -5.3267e-01,
       -5.5565e+00,  8.4429e-01,  4.9819e+00, -4.6006e-01,  2.2781e+00,
       -1.0639e+00,  1.2235e+00, -4.5606e+00,  3.4419e-01,  1.9167e+00,
       -1.5078e+00,  3.6443e+00,  2.8392e+00,  4.9837e-01, -5.5096e-02,
       -7.9940e+00, -1.9028e+00, -3.9536e+00,  3.7232e-03,  2.3186e+00,
        5.6297e+00,  2.2466e+00,  5.2483e-01,  1.2710e+00, -5.8875e-01,
       -2.2971e+00,  1.4909e-01,  2.7780e-03,  3.9281e+00,  6.5166e-01,
        8.9464e-02,  3.2467e+00,  9.5456e-01, -2.4231e+00, -4.38

On peut même résoudre des problèmes d'analogie. Par ex., qu'est-ce qui est à "femme" ce que "homme" est à "roi" ? Ou la relation "capitale de".

La fonction *close_words_from_vector* retourne les *k* mots les plus similaires au vecteur *vec* donné en entrée.

In [11]:
def close_words_from_vector(vec, k=10):
    ms = nlp.vocab.vectors.most_similar(np.array([vec]), n=k)
    return [nlp.vocab.strings[w] for w in ms[0][0]]

L'opération est : v_roi - v_homme + v_femme (cf. cours) :

In [12]:
analogie = nlp("roi").vector-nlp("homme").vector+nlp("femme").vector
close_words_from_vector(analogie)

['roi',
 'reine',
 'Roi',
 'prince',
 'régent',
 'duc',
 'princesse',
 'monarque',
 'suzerain',
 'coempereur']

In [13]:
analogie = nlp("france").vector-nlp("paris").vector+nlp("berlin").vector
close_words_from_vector(analogie)

['allemagne',
 'europe',
 'france',
 'amérique',
 'l´Allemagne',
 'afrique',
 'dafrique',
 'espagne',
 'autriche',
 'lafrique']

In [14]:
analogie = nlp("musique").vector-nlp("mozart").vector+nlp("picasso").vector
close_words_from_vector(analogie)

['musique',
 'pusique',
 'Aramusique',
 'photographie',
 'sculture',
 'culture',
 'kulture',
 'contre-culture',
 'musicalité',
 'contemporaine']

On peut aussi travailler à partir de plusieurs mots.

In [18]:
#ph = nlp("science").vector
ph = nlp("données science").vector
close_words_from_vector(ph)

['données',
 'd‘informations',
 'quantifications',
 'nformations',
 'informations',
 'connaissances',
 'd´informations',
 'identifications',
 'désinformations',
 'authentifications']

Pour finir, lien vers une visualisation intéressante :

http://projector.tensorflow.org